In [5]:
import openai

from scapy.utils import RawPcapReader
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP


In [6]:
def process_pcap(file_name):
    print('Opening {}...'.format(file_name))

    count = 0
    interesting_packet_count = 0

    interesting_packets = []
    for (pkt_data, pkt_metadata,) in RawPcapReader(file_name):
        count += 1

        ether_pkt = Ether(pkt_data)
        if 'type' not in ether_pkt.fields:
            # LLC frames will have 'len' instead of 'type'.
            # We disregard those
            continue

        if ether_pkt.type != 0x0800:
            # disregard non-IPv4 packets
            continue
        ip_pkt = ether_pkt[IP]
        if ip_pkt.proto != 1:
            # Ignore non-TCP packet
            continue
        interesting_packet_count += 1
        interesting_packets.append(ether_pkt)

    print('{} contains {} packets ({} interesting)'.
          format(file_name, count, interesting_packet_count))

    return interesting_packets


In [7]:
packets = process_pcap("../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng")

Opening ../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng...
../pcap-analysis/data/encoder/pcap/custom_pings3.pcapng contains 2261 packets (2261 interesting)


In [10]:
packets[1].show()

###[ Ethernet ]### 
  dst       = 84:a9:38:6a:7f:0d
  src       = 70:db:98:81:93:40
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 0
     flags     = 
     frag      = 0
     ttl       = 109
     proto     = icmp
     chksum    = 0xefc7
     src       = 8.8.8.8
     dst       = 130.231.202.234
     \options   \
###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0x1393
        id        = 0x2
        seq       = 0x1
        unused    = ''
###[ Raw ]### 
           load      = '\\xb5H\\x87c\x00\x00\x00\x00\\xee\\xea\x02\x00\x00\x00\x00\x00\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./01234567'



In [2]:
engines = openai.Engine.list()

In [11]:
network_prompt = """
This is a ping packet:

###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 38219
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = icmp
     chksum    = 0x477c
     src       = 130.231.202.234
     dst       = 8.8.8.8
     \options   \
###[ ICMP ]### 
        type      = echo-request
        code      = 0
        chksum    = 0xb93
        id        = 0x2
        seq       = 0x1
        unused    = ''
        
This is another ping packet:

###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 0
     flags     = 
     frag      = 0
     ttl       = 109
     proto     = icmp
     chksum    = 0xefc7
     src       = 8.8.8.8
     dst       = 130.231.202.234
     \options   \
###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0x1393
        id        = 0x2
        seq       = 0x1
        unused    = ''
        
Generate 5 new ping packets.
"""

In [15]:
completion = openai.Completion.create(engine="text-davinci-003", prompt=network_prompt,max_tokens=1500)

In [17]:
print(completion.choices[0].text)
packet_strs = 


1) ###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 63214
     flags     = 
     frag      = 0
     ttl       = 128
     proto     = icmp
     chksum    = 0x25ce
     src       = 192.168.10.67
     dst       = 10.20.30.45
     \options   ###[ ICMP ]### 
        type      = echo-request
        code      = 0
        chksum    = 0x973
        id        = 0x4
        seq       = 0x2
        unused    = '' 
        
2) ###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 84
     id        = 33244
     flags     = 
     frag      = 0
     ttl       = 32
     proto     = icmp
     chksum    = 0xa032
     src       = 172.16.4.10
     dst       = 11.22.33.44
     \options   ###[ ICMP ]### 
        type      = echo-reply
        code      = 0
        chksum    = 0x2b05
        id        = 0x3
        seq       = 0x1
        unused    = '' 
        
3) ###[ IP ]### 
     version   = 4
     ihl     